## 1. Instalación de Dependencias

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
import torch
import unsloth

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Unsloth version: {unsloth.__version__}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
VRAM Total: 15.83 GB
Unsloth version: 2026.1.4


In [ ]:
import torch
import gc
# Liberar memoria de Python
gc.collect()
# Vaciar caché de la GPU
torch.cuda.empty_cache()

## 2. Carga del Modelo Base con Unsloth

Cargamos Qwen3-8B en formato 4-bit (QLoRA) para reducir uso de VRAM.

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

# Cargar modelo y tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

print(f"Modelo cargado exitosamente!")
print(f"Parámetros del modelo: {model.num_parameters() / 1e9:.2f}B")

==((====))==  Unsloth 2026.1.4: Fast Qwen3 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Modelo cargado exitosamente!
Parámetros del modelo: 8.19B


## 3. Configuración de LoRA

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# Ver parámetros entrenables
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
print(f"Parámetros entrenables: {trainable_params:,} ({100 * trainable_params / all_params:.2f}%)")
print(f"Parámetros totales: {all_params:,}")

Unsloth 2026.1.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


Parámetros entrenables: 87,293,952 (1.65%)
Parámetros totales: 5,279,101,952


## 4. Preparación del Dataset

In [ ]:
alpaca_prompt = """A continuación hay una instrucción que describe una tarea, junto con una entrada que proporciona más contexto. Escribe una respuesta que complete apropiadamente la solicitud.

### Instrucción:
{}

### Entrada:
{}

### Respuesta:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    """Formatea ejemplos en el formato Alpaca."""
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="dataset_finetuning.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

print(f"Dataset preparado: {len(dataset)} ejemplos")
print("\nEjemplo de formato:")
print(dataset[0]["text"][:1000] + "...")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/513 [00:00<?, ? examples/s]

Dataset preparado: 513 ejemplos

Ejemplo de formato:
A continuación hay una instrucción que describe una tarea, junto con una entrada que proporciona más contexto. Escribe una respuesta que complete apropiadamente la solicitud.

### Instrucción:
Responde como un asesor de atención al cliente, de forma clara, profesional y orientada a resolver la consulta.

### Entrada:
¿Qué es un impuesto?

### Respuesta:
Es un tributo cuyo pago no origina una contraprestación directa en favor del contribuyente por parte del Estado. Un ejemplo de esto es el Impuesto a la Renta.<|im_end|>...


## 5. Configuración del Entrenamiento

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 10,
        num_train_epochs = 3,
        max_steps = -1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        gradient_checkpointing = True,
    ),
)

print("Trainer configurado. Iniciando entrenamiento...")
print(f"Batch size efectivo: {2 * 4} (per_device * gradient_accumulation)")
print(f"Total steps: ~{len(dataset) * 3 // (2 * 4)} steps")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/513 [00:00<?, ? examples/s]

Trainer configurado. Iniciando entrenamiento...
Batch size efectivo: 8 (per_device * gradient_accumulation)
Total steps: ~192 steps


## 6. Entrenamiento (Fine-tuning)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"VRAM en uso antes del training: {start_gpu_memory} GB.")

# Entrenar
trainer_stats = trainer.train()

# Estadísticas finales
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)

print(f"\n{'='*50}")
print(f"Entrenamiento completado!")
print(f"{'='*50}")
print(f"Tiempo total: {trainer_stats.metrics['train_runtime']:.2f} segundos")
print(f"VRAM peak usada: {used_memory} GB")
print(f"VRAM para LoRA: {used_memory_for_lora} GB")
print(f"Porcentaje usado: {used_percentage}%")
print(f"Loss final: {trainer_stats.metrics['train_loss']:.4f}")

The model is already on multiple devices. Skipping the move to device specified in `args`.


GPU = Tesla T4. Max memory = 14.741 GB.
VRAM en uso antes del training: 7.395 GB.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 513 | Num Epochs = 3 | Total steps = 195
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 87,293,952 of 8,278,029,312 (1.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.316300
2,2.449100
3,2.359600
4,2.266800
5,1.992900
6,2.013700
7,1.747900
8,1.756800
9,1.594000
10,1.427000



Entrenamiento completado!
Tiempo total: 1222.54 segundos
VRAM peak usada: 9.752 GB
VRAM para LoRA: 2.357 GB
Porcentaje usado: 66.156%
Loss final: 0.6780


## 7. Inferencia con el Modelo Fine-tuned

In [ ]:
FastLanguageModel.for_inference(model)

def generar_respuesta(instruccion, input_texto):
    """Genera una respuesta usando el modelo fine-tuned."""
    prompt = alpaca_prompt.format(
        instruccion,
        input_texto,
        ""  # Respuesta vacía, el modelo la completará
    )

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens = 1024,
        temperature = 0.4,
        top_p = 0.9,
        top_k = 50,
        use_cache = True
    )

    # Decodificar solo la respuesta generada (sin el prompt)
    resultado_completo = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraer solo la respuesta
    if "### Respuesta:" in resultado_completo:
        respuesta = resultado_completo.split("### Respuesta:")[1].strip()
    else:
        respuesta = resultado_completo

    return respuesta

print("Modelo listo para inferencia!")

Modelo listo para inferencia!


#Pruebas

In [ ]:
print("="*60)

consulta1 = "¿Qué es un impuesto?"
respuesta1 = generar_respuesta(
    "Responde como un asesor de atención al cliente, de forma clara, profesional y orientada a resolver la consulta.",
    consulta1
)

print(f"Cliente: {consulta1}")
print(f"\nAsistente: {respuesta1}")

Cliente: ¿Qué es un impuesto?

Asistente: Un impuesto es un tributo cuyo pago no origina una contraprestación directa en favor del contribuyente por parte del Estado.


In [ ]:
print("="*60)

consulta1 = "¿Qué son los tributos no vinculados?"
respuesta1 = generar_respuesta(
    "Responde como un asesor de atención al cliente, de forma clara, profesional y orientada a resolver la consulta.",
    consulta1
)

print(f"Cliente: {consulta1}")
print(f"\nAsistente: {respuesta1}")

Cliente: ¿Qué son los tributos no vinculados?

Asistente: Los tributos no vinculados son los impuestos.


## 8. Guardar el Modelo

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
save_path = "/content/drive/My Drive/Modelos_Qwen/Sunat_LoRA_v1"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("✅ Adaptadores LoRA guardados en: Qwen3-4B-Sunat-v1/")
print("Para cargar: model = FastLanguageModel.from_pretrained('Qwen3-4B-Sunat-v1')")

✅ Adaptadores LoRA guardados en: Qwen3-4B-Sunat-v1/
Para cargar: model = FastLanguageModel.from_pretrained('Qwen3-4B-Sunat-v1')


In [ ]:

model.save_pretrained_merged(
    "/content/drive/My Drive/Modelos_Qwen/Sunat_Merged_16bit",
    tokenizer,
    save_method = "merged_16bit",
)

print("✅ Modelo completo mergeado guardado en: qwen_QA_Sunat_merged/")
print("Este modelo puede ser cargado directamente con Transformers estándar")

In [ ]:
respuesta = preguntar_sunat("que es el RUC?")
print(respuesta)

El RUC es el Registro Único de Contribuyentes, el cual identifica a los contribuyentes del IR, NRUS y el Nuevo Régimen Único Simplificado.
